# 🏃‍♀️ Half Marathon Race Pacing / Allure Semi-Marathon

**🌐 Language / Langue:** Use the dropdown below to switch between English and French. / Utilisez le menu déroulant ci-dessous pour passer entre anglais et français.

---

### How to Use This Notebook / Comment Utiliser Ce Notebook

**English:**
1. **Set your target** - Enter your goal finish time (e.g., `01:45:00`) or target pace (e.g., `05:00` per km)
2. **Adjust power fade** - Use negative values for a stronger finish (negative split), positive for a faster start
3. **Set rest duration** - How long you plan to stop at each water station (in seconds)
4. **Click "Calculate Pacing"** - View your personalized pacing plan with split times, rest stop arrivals, and course-specific strategy tips
5. **Print or screenshot** - Use the Quick Reference section at the bottom for a race-day cheat sheet

**Français:**
1. **Définissez votre objectif** - Entrez votre temps cible (ex: `01:45:00`) ou allure cible (ex: `05:00` par km)
2. **Ajustez la gestion d'effort** - Valeurs négatives pour une fin plus forte (split négatif), positives pour un départ rapide
3. **Définissez la durée des ravitos** - Combien de temps vous prévoyez vous arrêter à chaque station (en secondes)
4. **Cliquez "Calculer"** - Consultez votre plan d'allure personnalisé avec les temps de passage, arrivées aux ravitos et conseils stratégiques
5. **Imprimez ou capturez** - Utilisez la section Référence Rapide en bas pour votre aide-mémoire du jour de course

---

**Course / Parcours:** 21.06 km with ~153m elevation gain / avec ~153m de dénivelé positif

**Rest stops at / Ravitaillements à:** 5.3 km, 9.1 km, 14.5 km

In [ ]:
import xml.etree.ElementTree as ET
import math
from dataclasses import dataclass
from typing import List, Tuple, Optional
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11

# =============================================================================
# TRANSLATIONS DICTIONARY - English/French
# =============================================================================

TRANSLATIONS = {
    'en': {
        'title': '🏃‍♀️ Half Marathon Race Pacing',
        'course': 'Course',
        'rest_stops_at': 'Rest stops at',
        'set_your_target': '🎯 Set Your Target',
        'target_finish_time': 'Target Finish Time',
        'target_avg_pace': 'Target Average Pace',
        'finish_time': 'Finish Time:',
        'pace': 'Pace:',
        'power_fade': 'Power Fade:',
        'rest_duration': 'Rest Duration:',
        'calculate': 'Calculate Pacing',
        'elapsed': 'Elapsed',
        'split_time': 'Split Time',
        'actual': 'Actual',
        'elev_change': 'Elev Δ',
        'rest': 'Rest',
        'finish': 'Finish',
        'km': 'km',
        'grade': 'Grade',
        'total': 'Total',
        'strategy_even': 'EVEN PACING',
        'strategy_positive': 'POSITIVE SPLIT',
        'strategy_negative': 'NEGATIVE SPLIT',
        'running_time': 'Running time',
        'rest_time': 'Rest time',
        'total_time': 'Total time',
        'avg_pace': 'Avg pace',
        'gap_pace': 'GAP pace',
        'split_analysis': 'Split Analysis',
        'first_half': '1st half',
        'second_half': '2nd half',
        'elevation': 'Elevation',
        'uphill_kms': 'Uphill kms',
        'downhill_kms': 'Downhill kms',
        'split': 'Split',
        'slower_2nd': 'slower 2nd half',
        'faster_2nd': 'faster 2nd half',
        'even': 'Even',
        'rest_stop_arrival': '🚰 Rest Stop Arrival Times',
        'km_splits': '📊 Kilometer Splits',
        'course_profile': 'Course Elevation Profile',
        'pace_per_km': 'Pace per Kilometer',
        'range': 'Range',
        'uphill': 'uphill (slower)',
        'downhill': 'downhill (faster)',
        'faster_start': 'faster start',
        'slower_finish': 'slower finish',
        'slower_start': 'slower start',
        'faster_finish': 'faster finish',
        'seconds_per_stop': 'Seconds per rest stop',
        'no_stops': 'no stops',
        'quick_ref': '📋 Quick Reference',
        'level': 'Level',
        'elite': 'Elite',
        'advanced': 'Advanced',
        'competitive': 'Competitive',
        'strong': 'Strong',
        'intermediate': 'Intermediate',
        'recreational': 'Recreational',
        'target': 'TARGET',
        'strategy': 'Strategy',
        'pace_adjustment': 'Pace adjustment',
        'stops': 'stops',
        'print_pocket': '🖨️ Print Pocket Card',
        'print_wrist': '🖨️ Print Wrist Band',
        'course_sections': '📍 Course Sections',
        'section': 'Section',
        'distance': 'Distance',
        'time': 'Time',
        'elev': 'Elev',
        'negative_faster_finish': 'Negative = stronger finish',
        'positive_faster_start': 'Positive = faster start',
        'zero_even_pace': '0 = even pace',
        'input_mode': 'Input Mode:',
        'pacing_tip': 'Pacing Tip',
    },
    'fr': {
        'title': '🏃‍♀️ Allure Semi-Marathon',
        'course': 'Parcours',
        'rest_stops_at': 'Ravitaillements à',
        'set_your_target': '🎯 Définissez votre objectif',
        'target_finish_time': 'Temps cible',
        'target_avg_pace': 'Allure moyenne cible',
        'finish_time': 'Temps:',
        'pace': 'Allure:',
        'power_fade': 'Gestion effort:',
        'rest_duration': 'Durée ravito:',
        'calculate': 'Calculer',
        'elapsed': 'Cumul',
        'split_time': 'Intervalle',
        'actual': 'Réelle',
        'elev_change': 'Déniv.',
        'rest': 'Ravito',
        'finish': 'Arrivée',
        'km': 'km',
        'grade': 'Pente',
        'total': 'Total',
        'strategy_even': 'ALLURE CONSTANTE',
        'strategy_positive': 'SPLIT POSITIF',
        'strategy_negative': 'SPLIT NÉGATIF',
        'running_time': 'Temps de course',
        'rest_time': 'Temps ravito',
        'total_time': 'Temps total',
        'avg_pace': 'Allure moy',
        'gap_pace': 'Allure GAP',
        'split_analysis': 'Analyse des splits',
        'first_half': '1ère moitié',
        'second_half': '2ème moitié',
        'elevation': 'Dénivelé',
        'uphill_kms': 'kms montée',
        'downhill_kms': 'kms descente',
        'split': 'Split',
        'slower_2nd': '2ème moitié plus lente',
        'faster_2nd': '2ème moitié plus rapide',
        'even': 'Égal',
        'rest_stop_arrival': '🚰 Temps aux Ravitaillements',
        'km_splits': '📊 Splits par Kilomètre',
        'course_profile': 'Profil Altimétrique',
        'pace_per_km': 'Allure par Kilomètre',
        'range': 'Plage',
        'uphill': 'montée (plus lent)',
        'downhill': 'descente (plus rapide)',
        'faster_start': 'départ rapide',
        'slower_finish': 'fin plus lente',
        'slower_start': 'départ lent',
        'faster_finish': 'fin plus rapide',
        'seconds_per_stop': 'Secondes par ravitaillement',
        'no_stops': 'sans arrêt',
        'quick_ref': '📋 Référence Rapide',
        'level': 'Niveau',
        'elite': 'Élite',
        'advanced': 'Avancé',
        'competitive': 'Compétitif',
        'strong': 'Confirmé',
        'intermediate': 'Intermédiaire',
        'recreational': 'Loisir',
        'target': 'OBJECTIF',
        'strategy': 'Stratégie',
        'pace_adjustment': 'Ajustement allure',
        'stops': 'arrêts',
        'print_pocket': '🖨️ Carte Poche',
        'print_wrist': '🖨️ Brassard',
        'course_sections': '📍 Sections du Parcours',
        'section': 'Section',
        'distance': 'Distance',
        'time': 'Temps',
        'elev': 'Déniv.',
        'negative_faster_finish': 'Négatif = fin plus rapide',
        'positive_faster_start': 'Positif = départ rapide',
        'zero_even_pace': '0 = allure constante',
        'input_mode': 'Mode saisie:',
        'pacing_tip': 'Conseil d\'Allure',
    }
}

# Current language (global)
CURRENT_LANG = 'en'

def t(key):
    """Get translated string for current language."""
    return TRANSLATIONS.get(CURRENT_LANG, {}).get(key, key)

def set_language(lang):
    """Set current language."""
    global CURRENT_LANG
    CURRENT_LANG = lang

# =============================================================================
# COURSE SECTIONS - Based on actual GPX elevation analysis
# 
# Elevation profile (from GPX analysis):
# - 0-3.1km: CLIMB +33m (net), the biggest sustained climb
# - 3.1-5.3km: DESCENT -34m to first rest stop  
# - 5.3-9.1km: CLIMB +23m rolling ascent to second rest stop
# - 9.1-12.5km: DESCENT -52m, the biggest descent of the race
# - 12.5-16.6km: CLIMB +28m, second major climb (rest stop 3 is mid-climb at 14.5km!)
# - 16.6-19.4km: DESCENT -39m, final descent
# - 19.4-21.1km: ROLLING -6m to finish
# =============================================================================

COURSE_SECTIONS = [
    {
        'start_km': 0,
        'end_km': 5.3,
        'name_en': 'The Opening Climb',
        'name_fr': "L'Ascension d'Ouverture",
        'strategy_en': 'Main climb (0-3km +55m), then recover on descent to rest stop',
        'strategy_fr': 'Montée principale (0-3km +55m), récupérez dans la descente',
        'pacing_en': 'DON\'T BANK TIME HERE. Accept slower pace on the climb (your GAP will be on target). Let the descent come naturally - don\'t sprint it. Arrive at rest 1 feeling controlled, not spent.',
        'pacing_fr': 'NE CHERCHEZ PAS À GAGNER DU TEMPS ICI. Acceptez un rythme plus lent dans la montée (votre GAP sera bon). Laissez la descente venir naturellement. Arrivez au ravito 1 maîtrisé, pas épuisé.',
        'icon': '⛰️'
    },
    {
        'start_km': 5.3,
        'end_km': 9.1,
        'name_en': 'The Rolling Ascent',
        'name_fr': "L'Ascension Roulante",
        'strategy_en': 'Steady climb with variation (+39m gain), conserve energy',
        'strategy_fr': 'Montée régulière avec variations (+39m), économisez',
        'pacing_en': 'This is THE CRITICAL SECTION. You\'ll be tempted to push after rest 1, but there\'s still 16km to go. Keep effort steady on the rollers. If you can\'t talk comfortably here, you\'re going too hard.',
        'pacing_fr': 'C\'est LA SECTION CRITIQUE. Vous serez tenté d\'accélérer après le ravito 1, mais il reste 16km. Gardez un effort constant. Si vous ne pouvez pas parler confortablement, vous allez trop vite.',
        'icon': '📈'
    },
    {
        'start_km': 9.1,
        'end_km': 14.5,
        'name_en': 'The Big Drop & Climb',
        'name_fr': 'La Grande Descente et Montée',
        'strategy_en': 'Enjoy the big descent (-52m), then start the second climb to rest',
        'strategy_fr': 'Profitez de la grande descente (-52m), puis attaquez la seconde montée',
        'pacing_en': 'Use the big descent for FREE SPEED but stay RELAXED - don\'t burn matches. When the climb starts at ~12.5km, dig in mentally. Rest stop 3 at 14.5km is MID-CLIMB - don\'t stop too long or you\'ll get cold legs.',
        'pacing_fr': 'Profitez de la grande descente pour de la VITESSE GRATUITE mais restez DÉTENDU. Quand la montée commence vers 12.5km, accrochez-vous mentalement. Le ravito 3 à 14.5km est EN PLEINE MONTÉE - pas de pause trop longue.',
        'icon': '📉'
    },
    {
        'start_km': 14.5,
        'end_km': 21.1,
        'name_en': 'The Final Push',
        'name_fr': 'La Poussée Finale',
        'strategy_en': 'Finish the climb, then descend (-39m) and sprint to the finish',
        'strategy_fr': 'Terminez la montée, descendez (-39m) et sprintez vers l\'arrivée',
        'pacing_en': 'You\'re past the worst! Finish the remaining ~2km of climb, then GRAVITY IS YOUR FRIEND on the -39m descent. Open up the stride. Last 2km is flat/rolling - leave everything on the course. This is what you trained for!',
        'pacing_fr': 'Le pire est passé! Finissez les ~2km de montée restants, puis la GRAVITÉ EST VOTRE AMIE dans la descente de -39m. Ouvrez la foulée. Les derniers 2km sont plats - donnez tout. C\'est pour ça que vous vous êtes entraîné!',
        'icon': '🏁'
    }
]

In [ ]:
# Language selector widget - displayed at top of notebook
language_selector = widgets.Dropdown(
    options=[('English 🇬🇧', 'en'), ('Français 🇫🇷', 'fr')],
    value='en',
    description='',
    layout=widgets.Layout(width='150px')
)

language_box = widgets.HBox([
    widgets.HTML('<span style="font-size: 16px;">🌐 Language / Langue:</span>'),
    widgets.HTML('&nbsp;&nbsp;'),
    language_selector,
])

def on_language_change_global(change):
    """Handle language selection change - updates global state and UI labels."""
    set_language(change['new'])
    # Update button labels if they exist
    if 'calculate_btn' in globals():
        calculate_btn.description = t('calculate')
        print_pocket_btn.description = t('print_pocket')
        print_wrist_btn.description = t('print_wrist')
        # Update input mode options
        current_val = input_mode.value
        was_finish_time = 'finish' in current_val.lower() or 'temps' in current_val.lower() or 'target' in current_val.lower()
        new_options = [t('target_finish_time'), t('target_avg_pace')]
        input_mode.options = new_options
        input_mode.value = new_options[0] if was_finish_time else new_options[1]

language_selector.observe(on_language_change_global, names='value')

display(language_box)

In [ ]:
@dataclass
class TrackPoint:
    lat: float
    lon: float
    elevation: float
    time: str
    distance_from_start: float = 0.0
    grade_percent: float = 0.0


def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def parse_gpx(filepath: str) -> List[TrackPoint]:
    tree = ET.parse(filepath)
    root = tree.getroot()
    ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}
    trackpoints = []
    cumulative_distance = 0.0
    prev_point = None
    for trkpt in root.findall('.//gpx:trkpt', ns):
        lat = float(trkpt.get('lat'))
        lon = float(trkpt.get('lon'))
        ele_elem = trkpt.find('gpx:ele', ns)
        elevation = float(ele_elem.text) if ele_elem is not None else 0.0
        time_elem = trkpt.find('gpx:time', ns)
        time = time_elem.text if time_elem is not None else ''
        if prev_point is not None:
            distance = haversine(prev_point.lat, prev_point.lon, lat, lon)
            cumulative_distance += distance
        point = TrackPoint(lat=lat, lon=lon, elevation=elevation, time=time, distance_from_start=cumulative_distance)
        trackpoints.append(point)
        prev_point = point
    return trackpoints


def smooth_elevation(trackpoints: List[TrackPoint], window_size: int = 3) -> List[TrackPoint]:
    if len(trackpoints) < window_size:
        return trackpoints
    smoothed = []
    half_window = window_size // 2
    for i, point in enumerate(trackpoints):
        start_idx = max(0, i - half_window)
        end_idx = min(len(trackpoints), i + half_window + 1)
        avg_elevation = sum(trackpoints[j].elevation for j in range(start_idx, end_idx)) / (end_idx - start_idx)
        smoothed_point = TrackPoint(lat=point.lat, lon=point.lon, elevation=avg_elevation, time=point.time, distance_from_start=point.distance_from_start)
        smoothed.append(smoothed_point)
    return smoothed

In [ ]:
def calculate_segment_grades(trackpoints: List[TrackPoint], segment_size_m: float = 100) -> List[TrackPoint]:
    if len(trackpoints) < 2:
        return trackpoints
    for i in range(1, len(trackpoints)):
        distance_diff = trackpoints[i].distance_from_start - trackpoints[i-1].distance_from_start
        elevation_diff = trackpoints[i].elevation - trackpoints[i-1].elevation
        if distance_diff > 0:
            grade = (elevation_diff / distance_diff) * 100
        else:
            grade = 0.0
        trackpoints[i].grade_percent = grade
    window = 5
    grades = [p.grade_percent for p in trackpoints]
    for i in range(len(trackpoints)):
        start = max(0, i - window // 2)
        end = min(len(grades), i + window // 2 + 1)
        trackpoints[i].grade_percent = sum(grades[start:end]) / (end - start)
    return trackpoints


def gap_factor(grade_percent: float) -> float:
    return 0.0021 * grade_percent**2 + 0.034 * grade_percent + 1


def calculate_gap_adjusted_distance(trackpoints: List[TrackPoint]) -> float:
    total_gap_distance = 0.0
    for i in range(1, len(trackpoints)):
        segment_distance = trackpoints[i].distance_from_start - trackpoints[i-1].distance_from_start
        avg_grade = (trackpoints[i].grade_percent + trackpoints[i-1].grade_percent) / 2
        gap_mult = gap_factor(avg_grade)
        total_gap_distance += segment_distance * gap_mult
    return total_gap_distance

In [ ]:
def format_time(minutes: float) -> str:
    total_seconds = int(minutes * 60)
    hours = total_seconds // 3600
    remaining = total_seconds % 3600
    mins = remaining // 60
    secs = remaining % 60
    if hours > 0:
        return f"{hours}:{mins:02d}:{secs:02d}"
    return f"{mins}:{secs:02d}"


def parse_time_input(time_str: str) -> float:
    parts = time_str.strip().split(':')
    if len(parts) == 2:
        return float(parts[0]) + float(parts[1]) / 60
    elif len(parts) == 3:
        return float(parts[0]) * 60 + float(parts[1]) + float(parts[2]) / 60
    else:
        raise ValueError(f"Invalid time format: {time_str}")


def parse_pace_input(pace_str: str) -> float:
    parts = pace_str.strip().split(':')
    if len(parts) == 2:
        return float(parts[0]) + float(parts[1]) / 60
    else:
        raise ValueError(f"Invalid pace format: {pace_str}")


def get_fade_multiplier(power_fade: float, km: float, total_distance_km: float) -> float:
    """Calculate pace multiplier based on power fade setting.
    
    Positive fade = faster first half, slower second half (positive split)
    Negative fade = slower first half, faster second half (negative split)
    
    Returns multiplier applied to BOTH GAP and actual pace.
    """
    if power_fade == 0:
        return 1.0
    
    halfway = total_distance_km / 2
    # Each unit of fade = 0.5% pace adjustment
    fade_factor = power_fade * 0.005  # 0.005 per unit (0.5%)
    
    if km <= halfway:
        # First half: positive fade = faster (multiplier < 1)
        return 1.0 - fade_factor
    else:
        # Second half: positive fade = slower (multiplier > 1)
        return 1.0 + fade_factor


def calculate_elevation_changes(trackpoints, start_km, end_km):
    """Calculate elevation gain and loss between two distances."""
    start_m = start_km * 1000
    end_m = end_km * 1000
    segment_points = [p for p in trackpoints if start_m <= p.distance_from_start <= end_m]
    
    if len(segment_points) < 2:
        return 0, 0
    
    gain = 0
    loss = 0
    for i in range(1, len(segment_points)):
        diff = segment_points[i].elevation - segment_points[i-1].elevation
        if diff > 0:
            gain += diff
        else:
            loss += abs(diff)
    
    return gain, loss


def calculate_segment_gap_factor(trackpoints, start_m, end_m):
    """
    Calculate the weighted average GAP factor for a segment.
    
    This matches how gap_adjusted_distance is calculated: sum of (distance × gap_mult)
    divided by actual distance. This ensures consistency with the total GAP calculation.
    """
    # Find all trackpoint pairs that overlap with this segment
    total_gap_weighted = 0.0
    total_distance = 0.0
    grades_in_segment = []
    elevations_in_segment = []
    
    for i in range(1, len(trackpoints)):
        pt_start = trackpoints[i-1].distance_from_start
        pt_end = trackpoints[i].distance_from_start
        
        # Check if this pair overlaps with our segment
        if pt_end <= start_m or pt_start >= end_m:
            continue
        
        # Calculate overlap
        overlap_start = max(pt_start, start_m)
        overlap_end = min(pt_end, end_m)
        overlap_distance = overlap_end - overlap_start
        
        if overlap_distance > 0:
            # Use average grade for this trackpoint pair
            avg_grade = (trackpoints[i].grade_percent + trackpoints[i-1].grade_percent) / 2
            gap_mult = gap_factor(avg_grade)
            total_gap_weighted += overlap_distance * gap_mult
            total_distance += overlap_distance
            grades_in_segment.append(avg_grade)
            elevations_in_segment.append(trackpoints[i].elevation)
    
    if total_distance > 0:
        weighted_gap_mult = total_gap_weighted / total_distance
        avg_grade = sum(grades_in_segment) / len(grades_in_segment) if grades_in_segment else 0
        avg_elevation = sum(elevations_in_segment) / len(elevations_in_segment) if elevations_in_segment else 0
    else:
        weighted_gap_mult = 1.0
        avg_grade = 0.0
        avg_elevation = 0.0
    
    return weighted_gap_mult, avg_grade, avg_elevation


def calculate_pacing(trackpoints, target_finish_time_min, rest_stops, total_distance_km, gap_adjusted_distance_m, power_fade=0.0, rest_duration_sec=30):
    """
    Calculate pacing based on GAP (Grade-Adjusted Pace) model.
    
    Rest stops are subtracted from target finish time before calculating pace.
    This means longer rests require faster running to hit the same finish time.
    """
    gap_adjusted_distance_km = gap_adjusted_distance_m / 1000
    rest_duration_min = rest_duration_sec / 60.0
    
    # Calculate total rest time and subtract from target
    total_rest_time_min = len(rest_stops) * rest_duration_min
    running_time_min = target_finish_time_min - total_rest_time_min
    
    if running_time_min <= 0:
        raise ValueError(f"Rest time ({format_time(total_rest_time_min)}) exceeds target finish time ({format_time(target_finish_time_min)})")
    
    # Base GAP pace using GAP-adjusted distance and RUNNING time (not total time)
    base_gap_pace = running_time_min / gap_adjusted_distance_km
    
    km_splits = []
    cumulative_time = 0.0
    
    for km in range(1, int(total_distance_km) + 1):
        start_dist = (km - 1) * 1000
        end_dist = km * 1000
        
        # Calculate weighted GAP factor for this km segment
        gap_mult, avg_grade, avg_elevation = calculate_segment_gap_factor(
            trackpoints, start_dist, end_dist
        )
        
        # Fade adjustment (applies to both GAP and actual pace)
        fade_mult = get_fade_multiplier(power_fade, km, total_distance_km)
        
        # GAP pace adjusted for fade (effort level varies by half)
        gap_pace = base_gap_pace * fade_mult
        
        # Actual pace adjusted for terrain (uphill = slower, downhill = faster)
        actual_pace = gap_pace * gap_mult
        
        segment_time = actual_pace  # For 1km segment, time = pace
        cumulative_time += segment_time
        
        km_splits.append({
            'km': km, 
            'actual_pace_min_km': actual_pace, 
            'gap_pace_min_km': gap_pace,
            'base_gap_pace': base_gap_pace,
            'gap_mult': gap_mult, 
            'fade_mult': fade_mult,
            'grade_percent': avg_grade,
            'elevation_m': avg_elevation, 
            'segment_time_min': segment_time, 
            'cumulative_time_min': cumulative_time
        })
    
    # Handle final partial km
    final_km = int(total_distance_km)
    remaining_distance_km = total_distance_km - final_km
    if remaining_distance_km > 0.01:
        start_dist = final_km * 1000
        end_dist = total_distance_km * 1000
        
        gap_mult, avg_grade, avg_elevation = calculate_segment_gap_factor(
            trackpoints, start_dist, end_dist
        )
        fade_mult = get_fade_multiplier(power_fade, total_distance_km, total_distance_km)
        gap_pace = base_gap_pace * fade_mult
        actual_pace = gap_pace * gap_mult
        segment_time = actual_pace * remaining_distance_km
        cumulative_time += segment_time
        km_splits.append({
            'km': round(total_distance_km, 2), 
            'actual_pace_min_km': actual_pace, 
            'gap_pace_min_km': gap_pace,
            'base_gap_pace': base_gap_pace,
            'gap_mult': gap_mult, 
            'fade_mult': fade_mult,
            'grade_percent': avg_grade,
            'elevation_m': avg_elevation, 
            'segment_time_min': segment_time, 
            'cumulative_time_min': cumulative_time
        })
    
    # Calculate rest stop data
    rest_stop_data = []
    prev_arrival_time = 0.0
    prev_distance = 0.0
    
    for stop_km in rest_stops:
        stop_time = 0.0
        for split in km_splits:
            if split['km'] >= stop_km:
                fraction = stop_km - int(stop_km)
                if fraction > 0:
                    stop_time = split['cumulative_time_min'] - split['segment_time_min'] * (1 - fraction)
                else:
                    stop_time = split['cumulative_time_min']
                break
            stop_time = split['cumulative_time_min']
        
        split_time = stop_time - prev_arrival_time
        split_distance = stop_km - prev_distance
        
        # Calculate elevation changes for this segment
        elev_gain, elev_loss = calculate_elevation_changes(trackpoints, prev_distance, stop_km)
        
        # Calculate paces for this segment
        if split_distance > 0:
            actual_pace_segment = split_time / split_distance
            segment_gap_pace = base_gap_pace * get_fade_multiplier(power_fade, (prev_distance + stop_km) / 2, total_distance_km)
        else:
            actual_pace_segment = 0
            segment_gap_pace = base_gap_pace
        
        rest_stop_data.append({
            'stop_number': len(rest_stop_data) + 1, 
            'distance_km': stop_km,
            'arrival_time_min': stop_time, 
            'elapsed_time_str': format_time(stop_time),
            'split_from_prev_min': split_time, 
            'split_distance_km': split_distance,
            'actual_pace_min_km': actual_pace_segment, 
            'gap_pace_min_km': segment_gap_pace,
            'elev_gain_m': elev_gain, 
            'elev_loss_m': elev_loss,
            'suggested_rest_min': rest_duration_min, 
            'departure_time_min': stop_time + rest_duration_min
        })
        prev_arrival_time = stop_time
        prev_distance = stop_km
    
    return {
        'km_splits': km_splits, 
        'rest_stops': rest_stop_data, 
        'target_gap_pace': base_gap_pace,
        'total_gap_distance_km': gap_adjusted_distance_km, 
        'calculated_finish_time_min': cumulative_time,
        'power_fade': power_fade,
        'rest_duration_sec': rest_duration_sec,
        'total_rest_time_min': total_rest_time_min,
        'running_time_min': running_time_min
    }

In [ ]:
def plot_elevation_profile(trackpoints, rest_stops, total_distance_km):
    fig, ax = plt.subplots(figsize=(12, 4))
    distances_km = [p.distance_from_start / 1000 for p in trackpoints]
    elevations = [p.elevation for p in trackpoints]
    ax.fill_between(distances_km, elevations, alpha=0.3, color='#2E86AB')
    ax.plot(distances_km, elevations, color='#2E86AB', linewidth=2)
    for stop in rest_stops:
        ax.axvline(x=stop, color='#E94F37', linestyle='--', linewidth=1.5, alpha=0.8)
        stop_idx = min(range(len(trackpoints)), key=lambda i: abs(trackpoints[i].distance_from_start / 1000 - stop))
        stop_elev = trackpoints[stop_idx].elevation
        ax.annotate(f'Rest {rest_stops.index(stop) + 1}\n{stop} km', xy=(stop, stop_elev),
                   xytext=(stop + 0.3, stop_elev + 8), fontsize=10, color='#E94F37', fontweight='bold')
    ax.set_xlabel('Distance (km)', fontsize=12)
    ax.set_ylabel(f'{t("elevation")} (m)', fontsize=12)
    ax.set_title(t('course_profile'), fontsize=14, fontweight='bold')
    ax.set_xlim(0, total_distance_km + 0.5)
    min_elev = min(elevations)
    max_elev = max(elevations)
    ax.text(0.02, 0.95, f'{t("range")}: {min_elev:.0f}m - {max_elev:.0f}m', transform=ax.transAxes, 
            fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    plt.tight_layout()
    return fig


def plot_pace_comparison(pacing_data, total_distance_km):
    fig, ax = plt.subplots(figsize=(12, 4))
    km_splits = pacing_data['km_splits']
    kms = [s['km'] for s in km_splits]
    actual_paces = [s['actual_pace_min_km'] for s in km_splits]
    gap_pace = pacing_data['target_gap_pace']
    colors = ['#E94F37' if p > gap_pace else '#2E86AB' for p in actual_paces]
    ax.bar(kms, [p * 60 for p in actual_paces], width=0.8, color=colors, alpha=0.7, label='Actual Pace')
    ax.axhline(y=gap_pace * 60, color='#1B998B', linestyle='-', linewidth=2.5, label=f'GAP Pace ({format_time(gap_pace)}/km)')
    ax.axvline(x=total_distance_km / 2, color='#888888', linestyle=':', linewidth=1.5, alpha=0.7)
    ax.set_xlabel(t('km').upper(), fontsize=12)
    ax.set_ylabel(f'{t("pace")} (sec/km)', fontsize=12)
    ax.set_title(t('pace_per_km'), fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.text(0.02, 0.95, f'Red: {t("uphill")} | Blue: {t("downhill")}', transform=ax.transAxes, fontsize=10, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    plt.tight_layout()
    return fig


def display_rest_stops_table(pacing_data, total_distance_km, trackpoints):
    """Display rest stop arrival times with elevation changes and pace info."""
    finish_time_min = pacing_data['calculated_finish_time_min']
    base_gap_pace = pacing_data['target_gap_pace']
    power_fade = pacing_data.get('power_fade', 0)
    
    html = f'''
    <style>
        .rest-table {{ border-collapse: collapse; width: 100%; font-family: -apple-system, BlinkMacSystemFont, sans-serif; margin-top: 15px; font-size: 12px; }}
        .rest-table th, .rest-table td {{ border: 1px solid #ddd; padding: 8px 6px; text-align: center; }}
        .rest-table th {{ background-color: #E94F37; color: white; }}
        .rest-table tr:nth-child(even) {{ background-color: #f9f9f9; }}
        .rest-table .split-col {{ background-color: #fff3cd; }}
        .rest-table .finish-row {{ background-color: #d4edda !important; font-weight: bold; }}
        .uphill {{ color: #E94F37; }}
        .downhill {{ color: #1B998B; }}
    </style>
    <h3>{t('rest_stop_arrival')}</h3>
    <table class="rest-table">
        <tr>
            <th>{t('rest')}</th>
            <th>Dist</th>
            <th>{t('elapsed')}</th>
            <th>{t('split_time')}</th>
            <th>{t('actual')}</th>
            <th>GAP</th>
            <th>{t('elev_change')}</th>
            <th>{t('rest')}</th>
        </tr>
    '''
    
    prev_arrival_time = 0.0
    prev_distance = 0.0
    
    for stop in pacing_data['rest_stops']:
        # Format split time
        if stop['split_distance_km'] > 0:
            split_info = f"{format_time(stop['split_from_prev_min'])}"
        else:
            split_info = "-"
        
        # Format elevation changes
        elev_gain = stop.get('elev_gain_m', 0)
        elev_loss = stop.get('elev_loss_m', 0)
        elev_str = f"<span class='uphill'>+{elev_gain:.0f}</span> / <span class='downhill'>-{elev_loss:.0f}</span>"
        
        html += f'''<tr>
            <td><strong>{stop['stop_number']}</strong></td>
            <td>{stop['distance_km']} km</td>
            <td>{stop['elapsed_time_str']}</td>
            <td class="split-col">{split_info}</td>
            <td>{format_time(stop.get('actual_pace_min_km', 0))}/km</td>
            <td>{format_time(stop.get('gap_pace_min_km', 0))}/km</td>
            <td>{elev_str}</td>
            <td>{int(stop['suggested_rest_min'] * 60)}s</td>
        </tr>'''
        
        prev_arrival_time = stop['arrival_time_min']
        prev_distance = stop['distance_km']
    
    # Calculate finish line segment data
    finish_split_dist = total_distance_km - prev_distance
    finish_split_time = finish_time_min - prev_arrival_time
    
    # Calculate elevation changes for final segment
    elev_gain_finish, elev_loss_finish = calculate_elevation_changes(trackpoints, prev_distance, total_distance_km)
    
    # Calculate actual pace for final segment (based on terrain)
    if finish_split_dist > 0:
        finish_actual_pace = finish_split_time / finish_split_dist
        # Get GAP multiplier for final segment
        finish_gap_mult, _, _ = calculate_segment_gap_factor(
            trackpoints, prev_distance * 1000, total_distance_km * 1000
        )
        # GAP pace = actual pace / gap_mult (reverse of normal calculation)
        finish_gap_pace = finish_actual_pace / finish_gap_mult
    else:
        finish_actual_pace = 0
        finish_gap_pace = base_gap_pace
        finish_gap_mult = 1.0
    
    # Format elevation for finish
    elev_str_finish = f"<span class='uphill'>+{elev_gain_finish:.0f}</span> / <span class='downhill'>-{elev_loss_finish:.0f}</span>"
    
    html += f'''<tr class="finish-row">
        <td>🏁 {t('finish')}</td>
        <td>{total_distance_km:.2f} km</td>
        <td>{format_time(finish_time_min)}</td>
        <td class="split-col">{format_time(finish_split_time)}</td>
        <td>{format_time(finish_actual_pace)}/km</td>
        <td>{format_time(finish_gap_pace)}/km</td>
        <td>{elev_str_finish}</td>
        <td>-</td>
    </tr>'''
    
    html += "</table>"
    display(HTML(html))


def display_splits_table(pacing_data, trackpoints):
    """Display kilometer splits with elevation gain/loss per km."""
    html = f'''
    <style>
        .splits-table {{ border-collapse: collapse; width: 100%; font-family: -apple-system, BlinkMacSystemFont, sans-serif; font-size: 11px; }}
        .splits-table th, .splits-table td {{ border: 1px solid #ddd; padding: 5px 4px; text-align: center; }}
        .splits-table th {{ background-color: #2E86AB; color: white; }}
        .splits-table tr:nth-child(even) {{ background-color: #f9f9f9; }}
        .uphill {{ color: #E94F37; }}
        .downhill {{ color: #1B998B; }}
        .halfway {{ background-color: #fff3cd !important; }}
    </style>
    <h3>{t('km_splits')}</h3>
    <table class="splits-table">
        <tr>
            <th>KM</th>
            <th>{t('actual')}</th>
            <th>GAP</th>
            <th>{t('grade')}</th>
            <th>{t('elev_change')}</th>
            <th>{t('total')}</th>
        </tr>
    '''
    halfway_km = max(s['km'] for s in pacing_data['km_splits']) / 2
    
    for split in pacing_data['km_splits']:
        grade_class = 'uphill' if split['grade_percent'] > 0 else 'downhill' if split['grade_percent'] < 0 else ''
        row_class = 'halfway' if abs(split['km'] - halfway_km) < 1 else ''
        
        # Get elevation gain/loss for this km segment
        km_start = int(split['km'] - 1) if split['km'] == int(split['km']) else int(split['km'])
        km_end = split['km']
        elev_gain, elev_loss = calculate_elevation_changes(trackpoints, km_start, km_end)
        elev_str = f"<span class='uphill'>+{elev_gain:.0f}</span> / <span class='downhill'>-{elev_loss:.0f}</span>"
        
        html += f'''<tr class="{row_class}">
            <td>{split['km']:.1f}</td>
            <td>{format_time(split['actual_pace_min_km'])}/km</td>
            <td>{format_time(split['gap_pace_min_km'])}/km</td>
            <td class="{grade_class}">{split['grade_percent']:+.1f}%</td>
            <td>{elev_str}</td>
            <td>{format_time(split['cumulative_time_min'])}</td>
        </tr>'''
    html += "</table>"
    display(HTML(html))


def display_course_sections(pacing_data, trackpoints, total_distance_km):
    """Display course sections with strategies, pacing tips, and segment data."""
    
    # Build section data from rest stops - this ensures consistency
    rest_stops = pacing_data['rest_stops']
    finish_time = pacing_data['calculated_finish_time_min']
    
    # Map sections to rest stop data for accurate times
    # Section 1 (0-5.3km): time = rest_stop 1 arrival
    # Section 2 (5.3-9.1km): time = rest_stop 2 arrival - rest_stop 1 arrival  
    # Section 3 (9.1-14.5km): time = rest_stop 3 arrival - rest_stop 2 arrival
    # Section 4 (14.5-21.1km): time = finish - rest_stop 3 arrival
    
    section_times = []
    section_paces = []
    section_distances = []
    
    for i, section in enumerate(COURSE_SECTIONS):
        start_km = section['start_km']
        end_km = min(section['end_km'], total_distance_km)
        distance = end_km - start_km
        section_distances.append(distance)
        
        if i == 0:
            # First section: use rest stop 1 arrival time
            section_time = rest_stops[0]['arrival_time_min']
            section_pace = rest_stops[0]['actual_pace_min_km']
        elif i == 1:
            # Second section: between rest stops 1 and 2
            section_time = rest_stops[1]['arrival_time_min'] - rest_stops[0]['arrival_time_min']
            section_pace = rest_stops[1]['actual_pace_min_km']
        elif i == 2:
            # Third section: between rest stops 2 and 3
            section_time = rest_stops[2]['arrival_time_min'] - rest_stops[1]['arrival_time_min']
            section_pace = rest_stops[2]['actual_pace_min_km']
        else:
            # Last section: from rest stop 3 to finish
            section_time = finish_time - rest_stops[2]['arrival_time_min']
            section_pace = section_time / distance if distance > 0 else 0
        
        section_times.append(section_time)
        section_paces.append(section_pace)
    
    # First display the summary table
    html = f'''
    <style>
        .sections-table {{ border-collapse: collapse; width: 100%; font-family: -apple-system, BlinkMacSystemFont, sans-serif; margin-top: 15px; }}
        .sections-table th {{ background-color: #2E86AB; color: white; padding: 10px; }}
        .sections-table td {{ padding: 10px; border-bottom: 1px solid #ddd; vertical-align: top; }}
        .section-name {{ font-weight: bold; font-size: 14px; }}
        .section-strategy {{ color: #666; font-style: italic; }}
        .uphill {{ color: #E94F37; }}
        .downhill {{ color: #1B998B; }}
    </style>
    <h3>{t('course_sections')}</h3>
    <table class="sections-table">
        <tr>
            <th>{t('section')}</th>
            <th>{t('distance')}</th>
            <th>{t('time')}</th>
            <th>{t('pace')}</th>
            <th>{t('elev')}</th>
            <th>{t('strategy')}</th>
        </tr>
    '''
    
    for i, section in enumerate(COURSE_SECTIONS):
        start = section['start_km']
        end = min(section['end_km'], total_distance_km)
        
        # Get section name and strategy based on language
        name = section[f'name_{CURRENT_LANG}']
        strategy = section[f'strategy_{CURRENT_LANG}']
        
        # Use pre-calculated times and paces
        section_time = section_times[i]
        section_pace = section_paces[i]
        
        # Calculate elevation changes
        elev_gain, elev_loss = calculate_elevation_changes(trackpoints, start, end)
        
        html += f'''<tr>
            <td><span class="section-name">{section['icon']} {name}</span></td>
            <td>{start:.1f} - {end:.1f} km</td>
            <td>{format_time(section_time)}</td>
            <td>{format_time(section_pace)}/km</td>
            <td><span class='uphill'>+{elev_gain:.0f}</span> / <span class='downhill'>-{elev_loss:.0f}</span></td>
            <td><span class="section-strategy">{strategy}</span></td>
        </tr>'''
    
    html += '</table>'
    display(HTML(html))
    
    # Now display detailed pacing tips for each section
    html_tips = f'''
    <style>
        .pacing-tips {{ margin-top: 20px; font-family: -apple-system, BlinkMacSystemFont, sans-serif; }}
        .pacing-tip-box {{ background-color: #f8f9fa; border-left: 4px solid #2E86AB; padding: 12px 15px; margin: 10px 0; border-radius: 0 8px 8px 0; }}
        .pacing-tip-title {{ font-weight: bold; color: #2E86AB; margin-bottom: 5px; }}
        .pacing-tip-text {{ color: #333; line-height: 1.5; }}
    </style>
    <div class="pacing-tips">
    <h4>💡 {t('pacing_tip')}</h4>
    '''
    
    for section in COURSE_SECTIONS:
        name = section[f'name_{CURRENT_LANG}']
        pacing = section[f'pacing_{CURRENT_LANG}']
        icon = section['icon']
        
        html_tips += f'''
        <div class="pacing-tip-box">
            <div class="pacing-tip-title">{icon} {name}</div>
            <div class="pacing-tip-text">{pacing}</div>
        </div>
        '''
    
    html_tips += '</div>'
    display(HTML(html_tips))


def display_printable_table(pacing_data, total_distance_km, table_type='pocket'):
    """
    Display a printable table for race day reference.
    
    Args:
        pacing_data: Pacing calculation results
        total_distance_km: Total course distance
        table_type: 'pocket' (wider) or 'wrist' (narrow)
    """
    if table_type == 'wrist':
        # Narrow format for wristband
        html = '''
        <div style="font-family: monospace; font-size: 10px;">
        <style>
            @media print {
                .wrist-table { font-size: 9px; }
                .no-print { display: none; }
                body { margin: 0; padding: 10px; }
            }
            .wrist-table { border-collapse: collapse; font-family: monospace; font-size: 10px; margin: 10px 0; }
            .wrist-table th, .wrist-table td { border: 1px solid #000; padding: 2px 4px; text-align: center; }
            .wrist-table th { background-color: #f0f0f0; }
        </style>
        <table class="wrist-table">
            <tr><th>km</th><th>Time</th><th>Pace</th></tr>
        '''
        # Add rows for rest stops only
        for stop in pacing_data['rest_stops']:
            html += f'''<tr>
                <td>{stop['distance_km']:.1f}</td>
                <td>{stop['elapsed_time_str']}</td>
                <td>{format_time(stop.get('actual_pace_min_km', 0))}</td>
            </tr>'''
        # Add finish
        html += f'''<tr style="font-weight: bold;">
            <td>{total_distance_km:.1f}</td>
            <td>{format_time(pacing_data['calculated_finish_time_min'])}</td>
            <td>-</td>
        </tr>'''
        html += '</table></div>'
    else:
        # Pocket format - more detail
        html = f'''
        <div style="font-family: monospace;">
        <style>
            @media print {{
                .pocket-table {{ font-size: 11px; }}
                .no-print {{ display: none; }}
                body {{ margin: 0; padding: 15px; }}
            }}
            .pocket-table {{ border-collapse: collapse; font-family: monospace; font-size: 12px; margin: 15px 0; }}
            .pocket-table th, .pocket-table td {{ border: 1px solid #000; padding: 4px 8px; text-align: center; }}
            .pocket-table th {{ background-color: #f0f0f0; }}
        </style>
        <table class="pocket-table">
            <tr>
                <th>{t('section')}</th>
                <th>km</th>
                <th>{t('elapsed')}</th>
                <th>{t('pace')}</th>
                <th>+/-m</th>
            </tr>
        '''
        for stop in pacing_data['rest_stops']:
            html += f'''<tr>
                <td>R{stop['stop_number']}</td>
                <td>{stop['distance_km']:.1f}</td>
                <td>{stop['elapsed_time_str']}</td>
                <td>{format_time(stop.get('actual_pace_min_km', 0))}/km</td>
                <td>+{stop.get('elev_gain_m', 0):.0f}/-{stop.get('elev_loss_m', 0):.0f}</td>
            </tr>'''
        # Finish
        html += f'''<tr style="font-weight: bold;">
            <td>FINISH</td>
            <td>{total_distance_km:.1f}</td>
            <td>{format_time(pacing_data['calculated_finish_time_min'])}</td>
            <td>-</td>
            <td>-</td>
        </tr>'''
        html += '</table></div>'
    
    display(HTML(html))

In [ ]:
# Load GPX data
GPX_FILE = 'WR-GPX-Semi-marathon-du-Finistere.gpx'
REST_STOPS = [5.3, 9.1, 14.5]
SMOOTHING_WINDOW = 5

raw_trackpoints = parse_gpx(GPX_FILE)
smoothed_trackpoints = smooth_elevation(raw_trackpoints, SMOOTHING_WINDOW)
trackpoints = calculate_segment_grades(smoothed_trackpoints)
total_distance_m = trackpoints[-1].distance_from_start
total_distance_km = total_distance_m / 1000
gap_adjusted_distance_m = calculate_gap_adjusted_distance(trackpoints)
gap_adjusted_distance_km = gap_adjusted_distance_m / 1000

In [ ]:
# Display elevation profile
fig_elevation = plot_elevation_profile(trackpoints, REST_STOPS, total_distance_km)
plt.show()

---
## 🎯 Set Your Target

In [ ]:
# Create widgets (language_selector is now at top of notebook)

# Input mode - use simple strings, will be updated by language change
input_mode = widgets.RadioButtons(
    options=['Target Finish Time', 'Target Average Pace'],
    value='Target Finish Time',
    description='',
    style={'description_width': 'initial'}
)

finish_time = widgets.Text(
    value='01:45:00',
    description='',
    placeholder='HH:MM:SS',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px')
)

avg_pace = widgets.Text(
    value='05:00',
    description='',
    placeholder='MM:SS per km',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px', display='none')
)

power_fade = widgets.IntSlider(
    value=0,
    min=-10,
    max=10,
    step=1,
    description='',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

rest_duration = widgets.IntSlider(
    value=30,
    min=0,
    max=120,
    step=5,
    description='',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

calculate_btn = widgets.Button(
    description='',
    button_style='success',
    icon='calculator',
    layout=widgets.Layout(width='200px', height='40px')
)

# Print buttons
print_pocket_btn = widgets.Button(
    description='',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

print_wrist_btn = widgets.Button(
    description='',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

output_area = widgets.Output()
printable_output = widgets.Output()

# Store last pacing data for print buttons
last_pacing_data = {'data': None}

def update_ui_language():
    """Update all UI elements with current language."""
    # Store current selection
    current_val = input_mode.value
    was_finish_time = 'finish' in current_val.lower() or 'temps' in current_val.lower()
    
    # Update options
    new_options = [t('target_finish_time'), t('target_avg_pace')]
    input_mode.options = new_options
    input_mode.value = new_options[0] if was_finish_time else new_options[1]
    
    calculate_btn.description = t('calculate')
    print_pocket_btn.description = t('print_pocket')
    print_wrist_btn.description = t('print_wrist')

# Initial UI setup
update_ui_language()

def toggle_inputs(change):
    finish_option = t('target_finish_time')
    if change['new'] == finish_option:
        finish_time.layout.display = 'block'
        avg_pace.layout.display = 'none'
    else:
        finish_time.layout.display = 'none'
        avg_pace.layout.display = 'block'

input_mode.observe(toggle_inputs, names='value')

def on_print_pocket(btn):
    """Display pocket card."""
    if last_pacing_data['data'] is None:
        return
    
    with printable_output:
        printable_output.clear_output()
        display(HTML(f'''
        <div style="margin: 10px 0;">
            <div style="border: 1px solid #ccc; padding: 15px; background: #f9f9f9; display: inline-block;">
                <pre style="font-family: monospace; font-size: 11px; margin: 0;">
Point   km      Time      Pace      Elev
─────────────────────────────────────────
R1      5.3     {last_pacing_data['data']['rest_stops'][0]['elapsed_time_str']}      {format_time(last_pacing_data['data']['rest_stops'][0].get('actual_pace_min_km', 0))}/km   +{last_pacing_data['data']['rest_stops'][0].get('elev_gain_m', 0):.0f}/-{last_pacing_data['data']['rest_stops'][0].get('elev_loss_m', 0):.0f}
R2      9.1     {last_pacing_data['data']['rest_stops'][1]['elapsed_time_str']}      {format_time(last_pacing_data['data']['rest_stops'][1].get('actual_pace_min_km', 0))}/km   +{last_pacing_data['data']['rest_stops'][1].get('elev_gain_m', 0):.0f}/-{last_pacing_data['data']['rest_stops'][1].get('elev_loss_m', 0):.0f}
R3      14.5    {last_pacing_data['data']['rest_stops'][2]['elapsed_time_str']}      {format_time(last_pacing_data['data']['rest_stops'][2].get('actual_pace_min_km', 0))}/km   +{last_pacing_data['data']['rest_stops'][2].get('elev_gain_m', 0):.0f}/-{last_pacing_data['data']['rest_stops'][2].get('elev_loss_m', 0):.0f}
FINISH  {total_distance_km:.1f}    {format_time(last_pacing_data['data']['calculated_finish_time_min'])}      -         -
─────────────────────────────────────────
                </pre>
            </div>
            <p style="font-size: 11px; color: #666; margin-top: 10px;">
                💡 Copy or screenshot for race day reference
            </p>
        </div>
        '''))

def on_print_wrist(btn):
    """Display wrist band."""
    if last_pacing_data['data'] is None:
        return
    
    with printable_output:
        printable_output.clear_output()
        display(HTML(f'''
        <div style="margin: 10px 0;">
            <div style="border: 1px solid #ccc; padding: 10px; background: #f9f9f9; display: inline-block;">
                <pre style="font-family: monospace; font-size: 10px; margin: 0;">
km     Time    Pace
────────────────────
5.3    {last_pacing_data['data']['rest_stops'][0]['elapsed_time_str']}    {format_time(last_pacing_data['data']['rest_stops'][0].get('actual_pace_min_km', 0))}
9.1    {last_pacing_data['data']['rest_stops'][1]['elapsed_time_str']}    {format_time(last_pacing_data['data']['rest_stops'][1].get('actual_pace_min_km', 0))}
14.5   {last_pacing_data['data']['rest_stops'][2]['elapsed_time_str']}    {format_time(last_pacing_data['data']['rest_stops'][2].get('actual_pace_min_km', 0))}
{total_distance_km:.1f}  {format_time(last_pacing_data['data']['calculated_finish_time_min'])}    -
────────────────────
                </pre>
            </div>
            <p style="font-size: 11px; color: #666; margin-top: 10px;">
                💡 Copy or screenshot for a wristband
            </p>
        </div>
        '''))

print_pocket_btn.on_click(on_print_pocket)
print_wrist_btn.on_click(on_print_wrist)

def on_calculate(btn):
    with output_area:
        clear_output(wait=True)
        try:
            # Check which input mode is selected
            finish_option = t('target_finish_time')
            if input_mode.value == finish_option:
                target_time_min = parse_time_input(finish_time.value)
            else:
                pace_min_km = parse_pace_input(avg_pace.value)
                target_time_min = pace_min_km * total_distance_km
            
            fade = power_fade.value
            rest_sec = rest_duration.value
            
            pacing_data = calculate_pacing(
                trackpoints, target_time_min, REST_STOPS,
                total_distance_km, gap_adjusted_distance_m, 
                power_fade=fade, rest_duration_sec=rest_sec
            )
            
            # Store for print buttons
            last_pacing_data['data'] = pacing_data
            
            # Summary
            print(f"\n{'='*50}")
            print(f"{t('target')}: {format_time(target_time_min)} finish time")
            print(f"{'='*50}")
            
            # Calculate first half vs second half using ACTUAL halfway point
            halfway_km = total_distance_km / 2
            first_half_splits = [s for s in pacing_data['km_splits'] if s['km'] <= halfway_km]
            second_half_splits = [s for s in pacing_data['km_splits'] if s['km'] > halfway_km]
            
            # Use actual distances for each half
            first_half_distance = halfway_km
            second_half_distance = total_distance_km - halfway_km
            
            # Calculate time for each half by interpolating at the exact halfway point
            first_half_time = 0.0
            for split in pacing_data['km_splits']:
                if split['km'] <= halfway_km:
                    first_half_time += split['segment_time_min']
                elif split['km'] > halfway_km:
                    # This split crosses the halfway point - take proportional part
                    km_start = int(split['km'] - 1) if split['km'] == int(split['km']) else int(split['km'])
                    if km_start < halfway_km:
                        # Calculate what fraction of this segment is in first half
                        fraction_in_first = halfway_km - km_start
                        first_half_time += split['actual_pace_min_km'] * fraction_in_first
                    break
            
            second_half_time = pacing_data['calculated_finish_time_min'] - first_half_time
            
            # Average paces per half
            first_half_pace = first_half_time / first_half_distance if first_half_distance > 0 else 0
            second_half_pace = second_half_time / second_half_distance if second_half_distance > 0 else 0
            
            # Calculate elevation gain/loss for each half
            elev_gain_1st, elev_loss_1st = calculate_elevation_changes(trackpoints, 0, halfway_km)
            elev_gain_2nd, elev_loss_2nd = calculate_elevation_changes(trackpoints, halfway_km, total_distance_km)
            
            # Count uphill vs downhill segments
            first_half_grades = [s['grade_percent'] for s in first_half_splits]
            second_half_grades = [s['grade_percent'] for s in second_half_splits]
            first_half_uphill = sum(1 for g in first_half_grades if g > 1)
            first_half_downhill = sum(1 for g in first_half_grades if g < -1)
            second_half_uphill = sum(1 for g in second_half_grades if g > 1)
            second_half_downhill = sum(1 for g in second_half_grades if g < -1)
            
            # Show strategy
            if fade != 0:
                fade_dir = t('strategy_positive') if fade > 0 else t('strategy_negative')
                fade_pct = abs(fade) * 0.5
                fade_desc = t('faster_start') if fade > 0 else t('slower_start')
                fade_desc2 = t('slower_finish') if fade > 0 else t('faster_finish')
                print(f"\n📍 {t('strategy')}: {fade_dir} ({fade:+d})")
                print(f"   {t('pace_adjustment')}: {fade_pct:.1f}% {fade_desc}, {fade_desc2}")
            else:
                print(f"\n📍 {t('strategy')}: {t('strategy_even')}")
            
            # Display times using pacing_data values
            total_rest_time = pacing_data['total_rest_time_min']
            running_time = pacing_data['running_time_min']
            
            print(f"\n⏱️  {t('running_time')}: {format_time(running_time)}")
            if total_rest_time > 0:
                print(f"   {t('rest_time')}: {format_time(total_rest_time)} ({len(REST_STOPS)} {t('stops')} × {rest_sec}s)")
                print(f"   {t('total_time')}: {format_time(target_time_min)}")
            print(f"   {t('avg_pace')}: {format_time(running_time / total_distance_km)}/km")
            print(f"   {t('gap_pace')}: {format_time(pacing_data['target_gap_pace'])}/km")
            
            print(f"\n📊 {t('split_analysis')}:")
            print(f"   {t('first_half')} ({first_half_distance:.1f}km): {format_time(first_half_time)} @ {format_time(first_half_pace)}/km")
            print(f"      {t('elevation')}: +{elev_gain_1st:.0f}m / -{elev_loss_1st:.0f}m | {t('uphill_kms')}: {first_half_uphill} | {t('downhill_kms')}: {first_half_downhill}")
            print(f"   {t('second_half')} ({second_half_distance:.1f}km): {format_time(second_half_time)} @ {format_time(second_half_pace)}/km")
            print(f"      {t('elevation')}: +{elev_gain_2nd:.0f}m / -{elev_loss_2nd:.0f}m | {t('uphill_kms')}: {second_half_uphill} | {t('downhill_kms')}: {second_half_downhill}")
            
            split_diff = second_half_time - first_half_time
            if split_diff > 0:
                print(f"\n   {t('split')}: +{format_time(abs(split_diff))} ({t('slower_2nd')})")
            elif split_diff < 0:
                print(f"\n   {t('split')}: -{format_time(abs(split_diff))} ({t('faster_2nd')})")
            else:
                print(f"\n   {t('split')}: {t('even')}")
            
            display(HTML('<br>'))
            
            # Display course sections with pacing tips
            display_course_sections(pacing_data, trackpoints, total_distance_km)
            
            display(HTML('<br>'))
            display_rest_stops_table(pacing_data, total_distance_km, trackpoints)
            display_splits_table(pacing_data, trackpoints)
            
            fig_pace = plot_pace_comparison(pacing_data, total_distance_km)
            plt.show()
            
            # Show quick reference section with print buttons inline
            display(HTML(f'''
            <div style="display: flex; align-items: center; margin-top: 20px; margin-bottom: 10px;">
                <h4 style="margin: 0;">🖨️ {t("quick_ref")}</h4>
            </div>
            '''))
            
            # Display print buttons
            display(widgets.HBox([print_pocket_btn, widgets.HTML('&nbsp;'), print_wrist_btn]))
            
            # Show pocket card by default
            with printable_output:
                printable_output.clear_output()
                on_print_pocket(None)
            
            display(printable_output)
            
        except Exception as e:
            import traceback
            print(f"Error: {e}")
            traceback.print_exc()
            print("Check your input format (HH:MM:SS or MM:SS)")

calculate_btn.on_click(on_calculate)

# Build UI labels based on current language
def build_ui():
    """Build the UI with current language settings."""
    return widgets.VBox([
        widgets.HTML(f'<label style="font-weight: bold;">{t("input_mode")}</label>'),
        input_mode,
        widgets.HTML('<br>'),
        widgets.HBox([
            widgets.HTML(f'<label style="width: 100px;">{t("finish_time")}</label>'),
            finish_time,
        ]),
        widgets.HBox([
            widgets.HTML(f'<label style="width: 100px;">{t("pace")}</label>'),
            avg_pace,
        ]),
        widgets.HTML('<br>'),
        widgets.HBox([
            widgets.HTML(f'<label style="width: 100px;">{t("power_fade")}</label>'),
            power_fade,
        ]),
        widgets.HTML(f'<div style="font-size: 11px; color: #666; margin-left: 105px;">{t("negative_faster_finish")} | {t("positive_faster_start")} | {t("zero_even_pace")}</div>'),
        widgets.HTML('<br>'),
        widgets.HBox([
            widgets.HTML(f'<label style="width: 100px;">{t("rest_duration")}</label>'),
            rest_duration,
        ]),
        widgets.HTML(f'<div style="font-size: 11px; color: #666; margin-left: 105px;">{t("seconds_per_stop")} ({t("no_stops")} = 0)</div>'),
        widgets.HTML('<br>'),
        calculate_btn,
        widgets.HTML('<br>'),
        output_area,
    ])

# Display interface
display(build_ui())

---
### 📋 Quick Reference / Référence Rapide

| Finish Time / Temps | Avg Pace / Allure | Level / Niveau |
|---------------------|-------------------|----------------|
| 1:30:00 | 4:16/km | Elite / Élite |
| 1:40:00 | 4:44/km | Advanced / Avancé |
| 1:45:00 | 5:00/km | Competitive / Compétitif |
| 1:50:00 | 5:14/km | Strong / Confirmé |
| 2:00:00 | 5:41/km | Intermediate / Intermédiaire |
| 2:15:00 | 6:24/km | Recreational / Loisir |

---

### 📐 About Grade Adjusted Pace (GAP) / À propos de l'Allure Ajustée au Dénivelé

**What is GAP?** GAP converts your actual pace on hills to an equivalent flat-ground pace, allowing you to compare effort across varying terrain. Running 5:00/km uphill requires more effort than 5:00/km on flat ground—GAP accounts for this.

**How it's calculated:** This tool uses a polynomial formula based on grade percentage:
```
GAP factor = 0.0021 × grade² + 0.034 × grade + 1.0
Actual Pace = GAP Pace × GAP factor
```

- **Uphill (+grade):** GAP factor > 1, so actual pace is slower than GAP pace
- **Downhill (-grade):** GAP factor < 1, so actual pace is faster than GAP pace  
- **Flat (0% grade):** GAP factor = 1, actual pace equals GAP pace

**Example:** At 5% uphill grade:
- GAP factor = 0.0021(25) + 0.034(5) + 1 = 1.22
- If your GAP pace is 5:00/km, your actual pace will be ~6:06/km
- But you're running at the same **effort level** as 5:00/km on flat ground

**Why it matters for this race:** With ~153m of elevation gain, your actual pace will vary significantly from flat to uphill sections. GAP helps you target a consistent **effort** rather than a consistent **pace**, which is the key to running strong on hilly courses.

---

**Qu'est-ce que le GAP?** Le GAP convertit votre allure réelle en côte en une allure équivalente sur terrain plat, permettant de comparer l'effort sur différents terrains.

**Pourquoi c'est important:** Avec ~153m de dénivelé positif, votre allure réelle variera considérablement. Le GAP vous aide à cibler un **effort** constant plutôt qu'une **allure** constante.